In [3]:
from importlib import reload
from gymnasium import spaces
import numpy as np
import logging
import random
import time
import copy

from Combined_Agent_Double.utils.actions import MORE_ACTIONS
from Combined_Agent_Double.Dueling_DDQN_Agent import Dueling_DDQN_Agent
from Combined_Agent_Double.utils.random_agent import RandomAgent
import Combined_Agent_Double.utils.stats_functions as sf
import hockey.hockey_env as h_env

SEED_TRAIN_1 = 7489
SEED_TRAIN_2 = 1312
seed = SEED_TRAIN_1

USE_MORE_ACTIONS = True

random.seed(seed)

reload(h_env)
env_name = "random_training_1"
env = h_env.HockeyEnv()

state_space = env.observation_space

if(USE_MORE_ACTIONS):
    action_space = spaces.Discrete(len(MORE_ACTIONS))
else: 
    action_space = env.discrete_action_space

agent = Dueling_DDQN_Agent(state_space, action_space, seed = seed, eps = 0.01, learning_rate = 0.0001, hidden_sizes = [256, 256], n_steps = 5)
agent.Q.load("more_actions_test_1_150k_30k_(256_256)")

opponent0 = RandomAgent(seed = seed)
opponent1 = h_env.BasicOpponent()
opponent2 = h_env.BasicOpponent(weak = False)
opponent3 = "Agent"

opponents = [opponent0, opponent1, opponent2, opponent3]
opponents_names = ["Random", "Weak", "NonWeak", "Dueling-DDQN Agent"]

match_history = [[] for _ in opponents]

stats = []
losses = []
betas = []
epsilons = []

frame_idx = 0

max_episodes = 1
games_to_play = 1
max_steps = 1

train_iterations = 32  # Number of training steps per episode

beta_start = agent.beta     # Beta annealing parameters
beta_frames = max_episodes * 10000       # Increase this to make annealing slower

time_start = time.time()        # Debugging

for episode in range(max_episodes):

    selected = random.randint(0, len(opponents) - 1)
    #opponent = opponents[selected]
    opponent = opponent3

    if(opponent == opponent3):
        opponent = copy.deepcopy(agent)

    for game in range(games_to_play):

        state, _ = env.reset(seed = seed)
        obs_agent2 = env.obs_agent_two()
        total_reward = 0

        for t in range(max_steps):

            frame_idx += 1
            
            agent.beta = min(1.0, beta_start + frame_idx * (1.0 - beta_start) / beta_frames)        # Smoother beta annealing

            done = False

            a1 = agent.perform_greedy_action(state)
            a2 = opponent.act(obs_agent2)
            
            print(state)
            print(obs_agent2)
            print(a1)
            print(a2)



Dueling DQN Network Architecture:
  Input Layer: (18)
  Hidden Layer 1: Linear(18, 256) -> Tanh
  Hidden Layer 2: Linear(256, 256) -> Tanh
  Value Stream: Linear(256, 1)
  Advantage Stream: Linear(256, 20)
------------------------------------------------------
Dueling DQN Network Architecture:
  Input Layer: (18)
  Hidden Layer 1: Linear(18, 256) -> Tanh
  Hidden Layer 2: Linear(256, 256) -> Tanh
  Value Stream: Linear(256, 1)
  Advantage Stream: Linear(256, 20)
------------------------------------------------------
Network loaded from more_actions_test_1_150k_30k_(256_256)/weights/training_finished.pth
[-3.          0.          0.          0.          0.          0.
  3.          0.          0.          0.          0.          0.
  1.34024     0.27389765  0.          0.          0.          0.        ]
[-3.         -0.          0.         -0.         -0.          0.
  3.         -0.          0.         -0.         -0.          0.
 -1.34024    -0.27389765 -0.         -0.          0.   